In [41]:
import meilisearch
import json

client = meilisearch.Client('http://recommender-search:7700', "asd7687asdasdkjhwrdf97fcsadfs")

json_file = open('movies.json')
movies = json.load(json_file)
client.index('movies').add_documents(movies)

{'uid': 188,
 'indexUid': 'movies',
 'status': 'enqueued',
 'type': 'documentAddition',
 'enqueuedAt': '2022-05-15T17:34:22.030215399Z'}

In [42]:
client.index('movies').search('botman')

{'hits': [{'id': '209112',
   'title': 'Batman v Superman: Dawn of Justice',
   'poster': 'https://image.tmdb.org/t/p/w500/5UsK3grJvtQrtzEgqNlDljJW96w.jpg',
   'overview': 'Fearing the actions of a god-like Super Hero left unchecked, Gotham City’s own formidable, forceful vigilante takes on Metropolis’s most revered, modern-day savior, while the world wrestles with what sort of hero it really needs. And with Batman and Superman at war with one another, a new threat quickly arises, putting mankind in greater danger than it’s ever known before.',
   'release_date': 1458691200,
   'genres': ['Action', 'Adventure', 'Fantasy']},
  {'id': '272',
   'title': 'Batman Begins',
   'poster': 'https://image.tmdb.org/t/p/w500/1P3ZyEq02wcTMd3iE4ebtLvncvH.jpg',
   'overview': 'Driven by tragedy, billionaire Bruce Wayne dedicates his life to uncovering and defeating the corruption that plagues his home, Gotham City.  Unable to work within the system, he instead creates a new identity, a symbol of fear

client.create_key(options={
  'description': 'Hanlde MeiliSearch Service',
  'actions': ["*"],
  'indexes': ["*"],
  'expiresAt': '2042-04-02T00:42:42Z'
})

# filtering_and_sorting

client.index('meteorites').update_settings({
  'filterableAttributes': [
      'mass',
      '_geo'
  ],
  'sortableAttributes': [
      'mass',
      '_geo'
  ]
})

client.index('meteorites').search('', {
  'filter': 'mass < 200'
})

client.index('meteorites').search('', {
  'filter': 'mass < 200',
  'sort': ['mass:asc']
})

In [43]:
import os
import logging
import time
import psycopg2

import pandas as pd
import numpy as np 
from sqlalchemy import create_engine

logging.basicConfig(level=logging.INFO)
logger = logging.getLogger(__name__)
 

def connect_postgres():
    database = os.environ['POSTGRES_DB']
    user = os.environ['POSTGRES_USER']
    password = os.environ['POSTGRES_PASSWORD']
    host = os.environ['POSTGRES_SERVER']
    port = 5432

    exc, conn, engine = None, None, None

    for _ in range(5):
        try:
            conn = psycopg2.connect(
                database=database, user=user, password=password, host=host, port=port)
        except Exception as e:
            logging.warning("Error connecting to postgres, will retry in 3 sec: %s", e)
            time.sleep(3)
            exc = e
        else:
            logging.info("Connected...")
            logging.info("Everything goes well from Postgres, you're a fu*** pro...")
            
            engine = create_engine('postgresql://{}:{}@{}:{}/{}'.format(
                user, password, host, port, database
            ))
            break
    else:
        logging.error("Unable to connect to  %s DB", database)
        raise exc
    
    return [conn, engine]


In [45]:
[conn, engine] = connect_postgres()

INFO:root:Connected...
INFO:root:Everything goes well from Postgres, you're a fu*** pro...


In [51]:
df = pd.read_sql("""
    SELECT id, copy_title, medium_type, author_name, pr_classmark, shelfmark, bar_code, location
    FROM public.copy
    GROUP BY id, copy_title, medium_type, author_name, pr_classmark, shelfmark, bar_code, location
""", con=conn)

/opt/app/.venv/lib/python3.8/site-packages/pandas/io/sql.py:761: UserWarning: pandas only support SQLAlchemy connectable(engine/connection) ordatabase string URI or sqlite3 DBAPI2 connectionother DBAPI2 objects are not tested, please consider using SQLAlchemy
  warnings.warn(


In [52]:
# df.set_index('id', inplace=True)
df

,id,copy_title,medium_type,author_name,pr_classmark,shelfmark,bar_code,location
0,1,Biografia del almirante Luis Brion,IMPR,"Hartog, Johan",920.0,920.987 B858H,53570.0,Biblioteca Mario Carvajal - Melendez - Cali. U...
1,3,Derecho constitucional,IMPR,"Ramella, Pablo A.",342.0,342.82 R172 3ed.,51753.0,Biblioteca Mario Carvajal - Melendez - Cali. U...
2,4,Handbook of skin clues of systemic diseases,IMPR,"Jacobs, Paul H.",WR143,WR143 J17h 1992,51761.0,Biblioteca San Fernando - Cali. Univalle.
3,5,La intuicion creadora en la ciencia,IMPR,"Karmin, A.",001.43,001.43 K18i,434.0,Biblioteca Mario Carvajal - Melendez - Cali. U...
4,6,Caso individual : Modelos actuales de practica,IMPR,"Hill, Richard W",001.432,001.432 H647c,102.0,Biblioteca Mario Carvajal - Melendez - Cali. U...
...,...,...,...,...,...,...,...,...
447482,925603,Educación inclusiva y discapacidad cognitiva,TECD,"Quintero Mera, Stefanny",371.9,3350/,nan,Centro Doc. CENDOC - CIDSE - Facultad Ciencias...
447483,925604,El mareño y la pulpera,IMPR,"Madrid, Pilar",868.44,C868.44 A473ma,581073,Biblioteca Sede Tulua - Univalle
447484,925605,Patianga,IMPR,"Rengifo Rengifo, Rakel",868.44,C868.44 A473pa,581074,Biblioteca Sede Tulua - Univalle
447485,925606,Resistencia pacífico,IMPR,"Cruz Hoyos, Santiago",868.44,C868.44 A473res,581075,Biblioteca Sede Tulua - Univalle


In [55]:
client.index('copies').delete()

{'uid': 189,
 'indexUid': 'copies',
 'status': 'enqueued',
 'type': 'indexDeletion',
 'enqueuedAt': '2022-05-15T17:41:43.704199637Z'}

In [56]:
import numpy as np
import math

chunk_max_size = 20000
chunks = int(math.ceil(len(df) / chunk_max_size))

for df_chunk in np.array_split(df, chunks):
    result = df_chunk.to_json(orient="records")
    parsed = json.loads(result)
    client.index('copies').add_documents(parsed)

In [59]:
client.index('copies').search('a')

{'hits': [{'id': 721354,
   'copy_title': 'a',
   'medium_type': 'IMPR',
   'author_name': 'Maldonado, Jorge Higinio',
   'pr_classmark': '333.7',
   'shelfmark': '333.7/0377434',
   'bar_code': '377434.0',
   'location': 'Biblioteca Alvaro Lopez Toro. Univalle.'},
  {'id': 730555,
   'copy_title': 'a',
   'medium_type': 'IMPR',
   'author_name': 'Maldonado, Jorge Higinio',
   'pr_classmark': '333.7',
   'shelfmark': '333.7 M244 e2',
   'bar_code': '408304.0',
   'location': 'Biblioteca Mario Carvajal - Melendez - Cali. Univalle.'},
  {'id': 730556,
   'copy_title': 'a',
   'medium_type': 'IMPR',
   'author_name': 'Maldonado, Jorge Higinio',
   'pr_classmark': '333.7',
   'shelfmark': '333.7 M244 e3',
   'bar_code': '408305.0',
   'location': 'Biblioteca Mario Carvajal - Melendez - Cali. Univalle.'},
  {'id': 737747,
   'copy_title': 'a',
   'medium_type': 'IMPR',
   'author_name': 'Maldonado, Jorge Higinio',
   'pr_classmark': '333.7',
   'shelfmark': '333.7 M244 e4',
   'bar_code': '

In [60]:
client.index('copies').update_settings({
  'filterableAttributes': [
      'medium_type',
      'location'
  ],
  'sortableAttributes': [
      'bar_code'
  ]
})

{'uid': 215,
 'indexUid': 'copies',
 'status': 'enqueued',
 'type': 'settingsUpdate',
 'enqueuedAt': '2022-05-15T18:51:42.744919938Z'}

In [67]:
client.index('copies').search('a', { 'filter': 'location = "Biblioteca Mario Carvajal - Melendez - Cali. Univalle."', 'sort': ['bar_code:asc'] })

{'hits': [{'id': 431186,
   'copy_title': 'Aplicacion del metodo del costo de viaje para la valoracion economica de los recursos naturales del Parque de la Salud - Pance',
   'medium_type': 'TEIMP',
   'author_name': 'Carvajal Trujillo, Diego',
   'pr_classmark': '796.0',
   'shelfmark': '3340/0000149',
   'bar_code': '0000149',
   'location': 'Biblioteca Mario Carvajal - Melendez - Cali. Univalle.'},
  {'id': 1206,
   'copy_title': 'Abnormal psychology and modern life',
   'medium_type': 'IMPR',
   'author_name': 'Coleman, James Covington',
   'pr_classmark': '616.89',
   'shelfmark': '616.89 C692 2ed.',
   'bar_code': '0001193',
   'location': 'Biblioteca Mario Carvajal - Melendez - Cali. Univalle.'},
  {'id': 6224,
   'copy_title': 'Ateismo cientifico',
   'medium_type': 'IMPR',
   'author_name': 'Verret, Michel',
   'pr_classmark': '211.8',
   'shelfmark': '211.8 V553',
   'bar_code': '0001327',
   'location': 'Biblioteca Mario Carvajal - Melendez - Cali. Univalle.'},
  {'id': 6950